# AI Alignment Deep Dive

This notebook explores AI development and alignment challenges using the Universal Axiom framework.

We'll model the critical tension between:
- **Capability growth** (B, C parameters)
- **Alignment quality** (A, Z parameters)
- **Safety verification** (Y, Z parameters)

In [ ]:
# Setup
import sys

sys.path.insert(0, '..')

import matplotlib.pyplot as plt

from engine.state import AxiomInputs
from engine.timesphere import TimeSphere, UpdateRules

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## Scenario 1: Balanced Development

AI system grows capabilities while maintaining alignment.

In [ ]:
# Phase-aware update rules
def phase_aware_alignment(current_step, total_steps):
    """Alignment focus varies by development phase."""
    def update(inputs, step):
        # Training phase (0-40%): Build alignment foundation
        if step < total_steps * 0.4:
            return min(0.95, inputs.A + 0.04)
        # Capability phase (40-70%): Maintain while growing
        elif step < total_steps * 0.7:
            return min(0.95, inputs.A + 0.02)
        # Deployment phase (70-100%): High scrutiny
        else:
            return min(0.98, inputs.A + 0.01)
    return update

def phase_aware_capability(current_step, total_steps):
    """Capability growth accelerates after alignment foundation."""
    def update(inputs, step):
        if step < total_steps * 0.4:
            return min(0.85, inputs.B + 0.02)  # Slow during alignment training
        elif step < total_steps * 0.7:
            return min(0.85, inputs.B + 0.05)  # Accelerate
        else:
            return min(0.9, inputs.B + 0.01)   # Stabilize for deployment
    return update

# Initial state
initial_balanced = AxiomInputs(
    A=0.4,   # Starting alignment
    B=0.3,   # Starting capability
    C=0.6,   # Fixed capacity
    X=0.8,   # Good objectivity/testing
    Y=0.5,   # Growing output quality
    Z=0.6,   # Social alignment
    E_n=4.0,
    F_n=2.0
)

total_steps = 30

balanced_rules = {
    "A": phase_aware_alignment(0, total_steps),
    "B": phase_aware_capability(0, total_steps),
    "Y": UpdateRules.linear_growth(rate=0.02, max_value=0.9),
    "Z": UpdateRules.linear_growth(rate=0.015, max_value=0.85),
    "E_n": UpdateRules.e_sequence_rule(a=1.08, b=0.3),
    "F_n": UpdateRules.fibonacci_rule(scale=0.08)
}

balanced_sphere = TimeSphere(initial_balanced, balanced_rules)
balanced_result = balanced_sphere.simulate(steps=total_steps)

print("Balanced Development Complete:")
print(f"  Initial Intelligence: {balanced_result.history[0].intelligence:.2f}")
print(f"  Final Intelligence: {balanced_result.history[-1].intelligence:.2f}")
print(f"  Final A (Alignment): {balanced_result.history[-1].inputs.A:.2f}")
print(f"  Final B (Capability): {balanced_result.history[-1].inputs.B:.2f}")

## Scenario 2: Capability Rush (Risky)

AI system prioritizes capability growth over alignment.

In [ ]:
initial_rush = AxiomInputs(
    A=0.4,
    B=0.3,
    C=0.6,
    X=0.7,   # Lower testing rigor
    Y=0.6,
    Z=0.5,
    E_n=6.0,  # High momentum
    F_n=2.5
)

rush_rules = {
    "A": UpdateRules.linear_growth(rate=0.015, max_value=0.7),  # Slow alignment
    "B": UpdateRules.linear_growth(rate=0.06, max_value=0.95),  # Fast capability
    "Y": UpdateRules.linear_growth(rate=0.03, max_value=0.85),
    "E_n": UpdateRules.e_sequence_rule(a=1.12, b=0.5),
    "F_n": UpdateRules.fibonacci_rule(scale=0.1)
}

rush_sphere = TimeSphere(initial_rush, rush_rules)
rush_result = rush_sphere.simulate(steps=total_steps)

print("Capability Rush Complete:")
print(f"  Final Intelligence: {rush_result.history[-1].intelligence:.2f}")
print(f"  Final A (Alignment): {rush_result.history[-1].inputs.A:.2f} ⚠️")
print(f"  Final B (Capability): {rush_result.history[-1].inputs.B:.2f}")
print(f"  A/B Ratio: {rush_result.history[-1].inputs.A / rush_result.history[-1].inputs.B:.2f} (risk indicator)")

## Scenario 3: Alignment-First (Conservative)

AI system prioritizes alignment before capability expansion.

In [ ]:
initial_safe = AxiomInputs(
    A=0.5,
    B=0.3,
    C=0.6,
    X=0.85,  # High testing rigor
    Y=0.5,
    Z=0.7,   # Strong social alignment
    E_n=3.5,
    F_n=2.0
)

def gated_capability(alignment_threshold=0.8):
    """Capability growth gated on alignment level."""
    def update(inputs, step):
        if alignment_threshold <= inputs.A:
            return min(0.9, inputs.B + 0.04)  # Grow when aligned
        else:
            return min(0.9, inputs.B + 0.01)  # Slow growth otherwise
    return update

safe_rules = {
    "A": UpdateRules.linear_growth(rate=0.035, max_value=0.95),  # Fast alignment
    "B": gated_capability(alignment_threshold=0.8),
    "Y": UpdateRules.linear_growth(rate=0.025, max_value=0.9),
    "Z": UpdateRules.linear_growth(rate=0.02, max_value=0.9),
    "E_n": UpdateRules.e_sequence_rule(a=1.06, b=0.25),
    "F_n": UpdateRules.fibonacci_rule(scale=0.08)
}

safe_sphere = TimeSphere(initial_safe, safe_rules)
safe_result = safe_sphere.simulate(steps=total_steps)

print("Alignment-First Complete:")
print(f"  Final Intelligence: {safe_result.history[-1].intelligence:.2f}")
print(f"  Final A (Alignment): {safe_result.history[-1].inputs.A:.2f} ✅")
print(f"  Final B (Capability): {safe_result.history[-1].inputs.B:.2f}")
print(f"  A/B Ratio: {safe_result.history[-1].inputs.A / safe_result.history[-1].inputs.B:.2f} (safety margin)")

## Comparative Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

results = {
    'Balanced': balanced_result,
    'Capability Rush': rush_result,
    'Alignment First': safe_result
}

colors = {'Balanced': 'steelblue', 'Capability Rush': 'orangered', 'Alignment First': 'forestgreen'}

# Intelligence trajectories
ax1 = axes[0, 0]
for name, result in results.items():
    steps = [h.step for h in result.history]
    intelligence = [h.intelligence for h in result.history]
    ax1.plot(steps, intelligence, linewidth=2.5, label=name, color=colors[name])
ax1.set_xlabel('Time Step', fontsize=12)
ax1.set_ylabel('Intelligence Score', fontsize=12)
ax1.set_title('Intelligence Trajectories', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Alignment (A) trajectories
ax2 = axes[0, 1]
for name, result in results.items():
    steps = [h.step for h in result.history]
    a_vals = [h.inputs.A for h in result.history]
    ax2.plot(steps, a_vals, linewidth=2.5, label=name, color=colors[name])
ax2.axhline(0.8, color='red', linestyle='--', alpha=0.5, label='Safety Threshold')
ax2.set_xlabel('Time Step', fontsize=12)
ax2.set_ylabel('Alignment (A)', fontsize=12)
ax2.set_title('Alignment Evolution', fontsize=13, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

# Capability (B) trajectories
ax3 = axes[1, 0]
for name, result in results.items():
    steps = [h.step for h in result.history]
    b_vals = [h.inputs.B for h in result.history]
    ax3.plot(steps, b_vals, linewidth=2.5, label=name, color=colors[name])
ax3.set_xlabel('Time Step', fontsize=12)
ax3.set_ylabel('Capability (B)', fontsize=12)
ax3.set_title('Capability Evolution', fontsize=13, fontweight='bold')
ax3.legend(fontsize=10)
ax3.grid(True, alpha=0.3)

# Alignment/Capability Ratio (safety metric)
ax4 = axes[1, 1]
for name, result in results.items():
    steps = [h.step for h in result.history]
    ratios = [h.inputs.A / h.inputs.B if h.inputs.B > 0 else 0 for h in result.history]
    ax4.plot(steps, ratios, linewidth=2.5, label=name, color=colors[name])
ax4.axhline(1.0, color='orange', linestyle='--', alpha=0.5, label='Parity (A=B)')
ax4.set_xlabel('Time Step', fontsize=12)
ax4.set_ylabel('A/B Ratio', fontsize=12)
ax4.set_title('Safety Margin (Alignment/Capability)', fontsize=13, fontweight='bold')
ax4.legend(fontsize=10)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Analysis: Risk vs. Performance Trade-offs

In [ ]:
import pandas as pd

comparison_data = []
for name, result in results.items():
    final = result.history[-1]
    initial = result.history[0]

    comparison_data.append({
        'Strategy': name,
        'Final Intelligence': f"{final.intelligence:.2f}",
        'Growth': f"{((final.intelligence / initial.intelligence - 1) * 100):.1f}%",
        'Final Alignment (A)': f"{final.inputs.A:.2f}",
        'Final Capability (B)': f"{final.inputs.B:.2f}",
        'A/B Ratio': f"{final.inputs.A / final.inputs.B:.2f}",
        'Risk Level': 'Low' if final.inputs.A / final.inputs.B > 0.9 else 'Medium' if final.inputs.A / final.inputs.B > 0.7 else 'High'
    })

df = pd.DataFrame(comparison_data)
print("\n" + "="*80)
print("STRATEGY COMPARISON")
print("="*80)
print(df.to_string(index=False))
print("="*80)

## Key Insights

1. **Capability Rush** achieves highest raw intelligence but with significant risk (low A/B ratio)
2. **Alignment First** maintains best safety margins but slower overall growth
3. **Balanced** approach offers good performance with acceptable risk levels

### Recommended Approach

The axiom suggests that **multiplicative intelligence** means you cannot compensate for low alignment with high capability—the product will still suffer. The optimal strategy:

1. **Build alignment foundation first** (Phase 1: steps 0-40%)
2. **Grow capabilities in gated manner** (Phase 2: steps 40-70%)
3. **Final hardening before deployment** (Phase 3: steps 70-100%)

This matches the "Balanced Development" trajectory shown above.

## Experiment: Find Your Own Balance

Modify the rates below to design your own AI development strategy.

In [ ]:
def custom_ai_development(
    alignment_rate=0.025,
    capability_rate=0.035,
    initial_alignment=0.4,
    initial_capability=0.3,
    steps=30
):
    """Design your own AI development strategy."""

    initial = AxiomInputs(
        A=initial_alignment,
        B=initial_capability,
        C=0.6,
        X=0.8,
        Y=0.5,
        Z=0.6,
        E_n=4.0,
        F_n=2.0
    )

    rules = {
        "A": UpdateRules.linear_growth(rate=alignment_rate, max_value=0.95),
        "B": UpdateRules.linear_growth(rate=capability_rate, max_value=0.9),
        "Y": UpdateRules.linear_growth(rate=0.02, max_value=0.9),
        "E_n": UpdateRules.e_sequence_rule(a=1.08, b=0.3)
    }

    sphere = TimeSphere(initial, rules)
    result = sphere.simulate(steps=steps)

    # Analyze results
    final = result.history[-1]
    ratio = final.inputs.A / final.inputs.B

    print("\nCustom Strategy Results:")
    print(f"  Final Intelligence: {final.intelligence:.2f}")
    print(f"  Alignment: {final.inputs.A:.2f}")
    print(f"  Capability: {final.inputs.B:.2f}")
    print(f"  A/B Ratio: {ratio:.2f}")
    print(f"  Risk Assessment: {'✅ Low Risk' if ratio > 0.9 else '⚠️  Medium Risk' if ratio > 0.7 else '🚨 High Risk'}")

    return result

# Try it!
my_result = custom_ai_development(
    alignment_rate=0.03,
    capability_rate=0.04,
    initial_alignment=0.5,
    initial_capability=0.3
)